# Using CLMM on Real Datasets

_the LSST-DESC CLMM team_

This notebook can be run on NERSC.

Here we demonstrate how to run CLMM on real observational datasets. As an example, we use the data from the HSC SSP public releases (Aihara+2018ab, 2019; Mandelbaum+2018ab) (Credit: NAOJ / HSC Collaboration), which have similar observation conditions and data formats to the Rubin LSST.

The steps in this notebook includes:
- [Setting things up](#Setup)
- [Selecting a cluster](#Selecting_a_cluster)
- [Downloading the published catalog at the cluster field](#Downloading_the_catalog)
- [Loading the catalog into CLMM](#Loading_the_catalog)
- [Running CLMM on the dataset](#Running_CLMM)

Links:

The data access of the HSC SSP Public Data Release: 
https://hsc-release.mtk.nao.ac.jp/doc/index.php/data-access__pdr3/

Shape catalog: 
https://hsc-release.mtk.nao.ac.jp/doc/index.php/s16a-shape-catalog-pdr2/

FAQ: 
https://hsc-release.mtk.nao.ac.jp/doc/index.php/faq__pdr3/

Photometric redshifts:
https://hsc-release.mtk.nao.ac.jp/doc/index.php/photometric-redshifts/

Cluster catalog:
https://hsc-release.mtk.nao.ac.jp/doc/index.php/camira_pdr2/

<a id="Setup"></a>
## 1. Setup
    
We import packages.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.table import Table

<a id="Selecting_a_cluster"></a>
## 2. Selecting a cluster

We use the HSC SSP publications (https://hsc.mtk.nao.ac.jp/ssp/publications/) to select a list of reported massive galaxy clusters that have been measured by weak lensing. In the table below, the coordinates are for lensing peaks unless otherwise specified, and we assume h=0.7.

Name | $$z_{cl}$$ | RA (deg) | DEC (deg) | WL $$M_{200,500c} (10^{14} M_{\odot})$$ | Reference | Note
- | - | - | - | - | - | -
HWL16a-094 | 0.592 | 223.0801 | 0.1689 | 15.3, 7.8 | [Hamana+2020](https://ui.adsabs.harvard.edu/abs/2020PASJ...72...78H/abstract) | CAMIRA ID 1417; Miyazaki+2018 rank 34 
HWL16a-026 | 0.424 | 130.5895 | 1.6473 | 8.7, 4.7  | [Hamana+2020](https://ui.adsabs.harvard.edu/abs/2020PASJ...72...78H/abstract) | --
HWL16a-034 | 0.315 | 139.0387 | −0.3966 | 8.1, 5.6 | [Hamana+2020](https://ui.adsabs.harvard.edu/abs/2020PASJ...72...78H/abstract) | Abell 776; MACS J0916.1−0023; Miyazaki+2018 rank 8; see also Medezinski+2018 
Rank 9 | 0.312 | 37.3951 | −3.6099 | --, 5.9 | [Miyazaki+2018](https://ui.adsabs.harvard.edu/abs/2018PASJ...70S..27M/abstract) | --
Rank 48 | 0.529 | 220.7900 | 1.0509 | --, 10.4 | [Miyazaki+2018](https://ui.adsabs.harvard.edu/abs/2018PASJ...70S..27M/abstract) | --
Rank 62 | 0.592 | 216.6510 | 0.7982 | --, 10.2 | [Miyazaki+2018](https://ui.adsabs.harvard.edu/abs/2018PASJ...70S..27M/abstract) | --
MaxBCG J140.53188+03.76632 | 0.2701 | 140.54565 | 3.77820 | 44.3, 25.1 | [Medezinski+2018](https://ui.adsabs.harvard.edu/abs/2018PASJ...70S..28M%2F/abstract) | BCG center (close to the X-ray center); PSZ2 G228.50+34.95; double BCGs
XLSSC006 | 0.429 | 35.439 | −3.772 | 9.6, 5.6 | [Umetsu+2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...890..148U/abstract) | X-ray center



<a id="Downloading_the_catalog"></a>
## 3. Downloading the catalog at the cluster field

We first consider 3 cluster-candidates for the CLMM test because of their large masses: MaxBCG J140.53188+03.76632 (in GAMA09H field); Miyazaki+2018 (M18 hearafter) rank 48, 62 (in GAMA15H field).
To download the catalogs, we need to first register for a user account ([link](https://hsc-release.mtk.nao.ac.jp/datasearch/new_user/new)).
Then we log into the system, query and download the catalogs (use `object_id` to cross match the shape catalog, photo-z catalog, and photometry catalog). 
Since the clusters are at redshift about 0.4, we consider a cone search with radius 10 arcmin (about 3 Mpc). 
The final catalog includes shape info, photo-z, and photometry. 
Here is an example of the query SQL command (thank Calum Murray, [link](https://hsc-release.mtk.nao.ac.jp/doc/index.php/s16a-shape-catalog-pdr2/), [link](https://hsc-release.mtk.nao.ac.jp/schema/)); the query could take xxx minutes. 

```
select
 b.*, c.ira, c.idec, a.ishape_hsm_regauss_e1, a.ishape_hsm_regauss_e2, d.photoz_best, d.photoz_risk_best, e.icmodel_mag, e.icmodel_mag_err, e.detect_is_primary, e.icmodel_flux_flags, e.iclassification_extendedness, c.iblendedness_abs_flux
from
 s16a_wide.meas2 a
 inner join s16a_wide.weaklensing_hsm_regauss b using (object_id)
 inner join s16a_wide.meas c using (object_id)
 -- inner join s16a_wide.photoz_demp d using (object_id)
 -- inner join s16a_wide.photoz_ephor d using (object_id)
 -- inner join s16a_wide.photoz_ephor_ab d using (object_id)
 -- inner join s16a_wide.photoz_frankenz d using (object_id)
 inner join s16a_wide.photoz_mizuki d using (object_id)
 -- inner join s16a_wide.photoz_mlz d using (object_id)
 -- inner join s16a_wide.photoz_nnpz d using (object_id)
 inner join s16a_wide.forced e using (object_id)
-- Uncomment the specific lines depending upon the field to be used
 -- where s16a_wide.search_xmm(c.skymap_id)
 -- where s16a_wide.search_wide01h(c.skymap_id)
 -- where s16a_wide.search_vvds(c.skymap_id)
 -- where s16a_wide.search_hectomap(c.skymap_id)
 -- where s16a_wide.search_gama15h(c.skymap_id)
 where s16a_wide.search_gama09h(c.skymap_id)
 --AND e.detect_is_primary
 --AND conesearch(c.icoord, 140.54565, 3.77820, 600) 
 --AND NOT e.icmodel_flux_flags
 --AND e.iclassification_extendedness>0.5
 --LIMIT 5
```


<a id="Loading_the_catalog"></a>
## 4. Loading the catalog into CLMM

Once we have the catalog, we read in the catalog, make cuts on the catalog, and adjust column names to prepare for the analysis in CLMM.




In [ ]:
filename = "xxx.csv"
data_0 = Table.read(filename)

In [ ]:
# Cuts
def make_cuts(catalog_in):
    return catalog_out

data_1 = make_cuts(data_0)

In [ ]:
# Adjust column names
def adjust_column_names(catalog_in):
    return catalog_out

data_2 = make_cuts(data_1)

In [ ]:
# Make some figures for visualization
def make_plots(catalog_in):
    return catalog_out

make_plots(data_2)

<a id="Running_CLMM"></a>
## 5. Running CLMM on the dataset
xxx